#### 1 切换目录

In [6]:
%cd /home/workspace/vits_for_chinese  # 具体目录看实际情况调整
%ls

/home/featurize/vits-zh
attentions.py  logs/              monotonic_align/  text/          wave_data/
commons.py     losses.py          preprocess.py     train.py
configs/       mel_processing.py  __pycache__/      transforms.py
data_utils.py  models.py          requirements.txt  usage.txt
filelists/     modules.py         test.py           utils.py


#### 2 训练模型

In [ ]:
import utils
from train import Trainer

hps = utils.get_hparams_from_file('configs/config.json')
trainer = Trainer(hps)

trainer.train()

#### 3.模型推理

In [ ]:
import utils
import torch
import text
from models import VITS_Model
from IPython.display import display, Audio

hps = utils.get_hparams_from_file('configs/config.json')
model = VITS_Model(**hps.model)
model.load_state_dict(torch.load('logs/model/epoch_latest/generator.ckpt', map_location="cpu"))
model.eval();

In [ ]:
speaker_emb = torch.load('dataset/embed/wav_DLC_Story_CN/AUDIO_DLC_Story_CN 00003.emb.pt').unsqueeze(0)

In [ ]:
in_text = "这些尝试大部分集中于通用LlM上"
phonemes = text.pypinyin_g2p(in_text)
input_ids = torch.LongTensor(text.tokens2ids(phonemes)).unsqueeze(0)
lengths = torch.LongTensor([input_ids.size(1)])
t_audio = model.infer(input_ids, lengths, speaker_emb)
audio = t_audio[0][0,0].data.cpu().float().numpy()
display(Audio(audio, rate=hps.data.sampling_rate))

In [ ]:
# (in_text, sid, noise_scale=0.3, noise_scale_w=0.3, length_scale=1.0)